### **마운트**


*   Google Colab에 마운트합니다.



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


### **라이브러리 및 모듈 로드**

*   필요한 라이브러리들을 전부 설치하고 모듈을 로드합니다.




In [2]:
!apt-get update &> /dev/null
!apt install chromium-chromedriver &> /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin &> /dev/null
!pip install selenium &> /dev/null

from bs4 import BeautifulSoup
import requests
import urllib.request
from selenium import webdriver
import pandas as pd
import re
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

### **전역변수와 리스트 설정**

*   마지막 DataFrame에 들어갈 list들을 전역변수로 설정합니다.
*   키워드를 '조작'으로 설정한 이유는 최근 사회에서 다양한 조작 이슈(ex : 확률 조작, 언론 조작, 시세 조작...) 등이 있고, 뉴스 기사 크롤링을 통해 시간적 흐름에 따른 이슈의 변화 및 조작과 관련된 다양한 topic들을 추출할 수 있을 것 같아 선택하였습니다.
*   추출 기간은 2021년으로 설정하였고, 4월 1일까지의 기사를 추출하기 위해 두 전역변수를 설정하였습니다. 이 두 변수들은 검색 페이지의 url에서 쓰입니다.





In [3]:
url_list = []
title_list = []
date_list = []
content_list = []

keyword = '조작'
start_date = '20210101'
end_date = '20210401'

### **검색페이지 URL완성과 기사링크 접근하는 함수 만들기 + a) 제목, b) 날짜 추가**

*   검색 페이지 url을 위에 상기한 변수들을 통해 들어가고, 기사 링크 및 제목, 날짜까지 같이 크롤링합니다.
*   한 페이지에 기사가 10개가 출력되므로, for문을 통해 (총 기사 개수 / 10) + 1 번을 돌면서 필요한 정보들을 가져옵니다.
*   날짜는 날짜와 기사를 올린 시간이 묶여서 출력되는데, 날짜 부분만 가져오기 위해 [1:13]을 설정합니다.





In [4]:
def get_content_url(num):
    total_pages = int(num / 10) + 1

    driver = webdriver.Chrome('chromedriver', options=options)

    for i in range(1, total_pages + 1):
        url = 'http://search.khan.co.kr/search.html?stb=khan&dm=5&q={}&pg={}&sort=1&d1={}~{}'.format(keyword, i, start_date, end_date)
        driver.get(url)
        driver.implicitly_wait(1)
        driver.maximize_window()
        driver.implicitly_wait(1)

        try:
            for j in range(1, 10 + 1):
                title_url_xpath = driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[2]/dl[{}]/dt/a'.format(j))
                date_xpath = driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[2]/dl[{}]/dt/span'.format(j))

                title_list.append(title_url_xpath.text)
                url_list.append(title_url_xpath.get_attribute('href'))
                date_list.append(date_xpath.text[1:13])
        
        except:
            break

    driver.close()


### **기사링크 크롤링하여 c) 본문을 리스트에 넣는 함수 만들기**

*   url_list에 있는 기사 링크들을 통해 본문의 내용을 가져옵니다.
*   tmp는 class를 art_body로 가지는 div를 가져오고, 그 tmp 내에서 find_all을 통해 본문을 담고 있는 부분을 가져옵니다. 반복문을 통해 tmp2 내에 있는 기사 본문들을 담고, 이를 content라는 변수를 통해 기사를 이어 붙이면서 리스트에 담습니다.



In [7]:
def get_content(url_list):
    driver = webdriver.Chrome('chromedriver', options=options)

    crawling_counter = 1

    for url in url_list:
        try:
            driver.get(url)
            driver.implicitly_wait(1)
            driver.maximize_window()
            driver.implicitly_wait(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            tmp = soup.find('div', class_ = 'art_body')
            tmp2 = tmp.find_all('p', 'content_text')

            content = ''
            for i in tmp2:
                content += i.text
        
            content_list.append(content)
        
        except:
            content_list.append(' ')

        if crawling_counter % 20 == 0:
            print('{} out of {} content crawling done...'.format(crawling_counter, len(url_list)))

        crawling_counter += 1

    driver.close()


### **전체 기사의 개수 구하는 함수 만들기**

*   검색 페이지를 보면 해당 카테고리에 대해 총 몇개의 기사가 있는지 나옵니다.
*   메인 함수에서 제일 먼저 실행되는 구간입니다.



In [6]:
def content_num(url):
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.get(url)
    driver.implicitly_wait(1)
    driver.maximize_window()
    driver.implicitly_wait(1)

    num = driver.find_element_by_xpath('//*[@id="container"]/div[2]/div[2]/h3/span')
    
    return int(re.findall('\d+', num.text)[0])


### **메인함수 실행**

*   전체 기사의 개수를 구하고, 링크 / 제목 / 날짜를 가져오고, 마지막으로 본문을 가져오는 함수를 실행합니다.
*   각 항목에 대해서 진행 상황을 간단하게 print문으로 출력하였습니다.



In [8]:
init_url = 'http://search.khan.co.kr/search.html?stb=khan&dm=5&q={}&pg=1&sort=1&d1={}~{}'.format(keyword, start_date, end_date)

print('Get Number of Contents...')
num_content = content_num(init_url)
print('Total number of contents : {}'.format(num_content))
print()

print('Get Title, URL, Date...')
get_content_url(num_content)
print('Title, URL, Date Crawling Done.')
print()

print('Get Content...')
get_content(url_list)
print('Total Crawling Done.')

Get Number of Contents...
Total number of contents : 225

Get Title, URL, Date...
Title, URL, Date Crawling Done.

Get Content...
20 out of 225 content crawling done...
40 out of 225 content crawling done...
60 out of 225 content crawling done...
80 out of 225 content crawling done...
100 out of 225 content crawling done...
120 out of 225 content crawling done...
140 out of 225 content crawling done...
160 out of 225 content crawling done...
180 out of 225 content crawling done...
200 out of 225 content crawling done...
220 out of 225 content crawling done...
Total Crawling Done.


### **저장한 리스트 DataFrame 작성**


*   각 list들을 DataFrame으로 묶는 구간입니다.



In [9]:
total = pd.DataFrame({'title' : title_list, 'date' : date_list, 'url' : url_list, 'content' : content_list})

### **불필요한 행(향이네 관련) DataFrame에서 제거**

*   경향신문 기사를 긁다 보면 간혹 '향이네'라는 곳의 기사가 가져와지는 경우가 있습니다.
*   http://h2.khan.co.kr/
*   '향이네'는 홈페이지의 구조가 기본 경향신문의 구조와 아예 다르고, 기사의 수도 매우 적습니다.
*   따라서 위의 본문에서 try, except문을 통해 에러가 발생하는 경우는 기사 본문을 ' '로 담도록 설정하였고, 그 행을 삭제하도록 하는 부분입니다.




In [10]:
unused = []

for i in range(len(total['content'])):
    if total['content'][i] == ' ':
        unused.append(i)

total = total.drop(index = unused, axis = 0)

### **본문 불필요한 내용 전처리**

*   먼저 영어, 숫자, 한글, 공백을 제외한 모든 기호를 제외합니다.
*   다음으로는 공백이 2번 이상인 경우가 있을 수 있다고 생각하여, 그것들을 모두 공백 한칸으로 처리합니다.
*   strip 함수를 통해 전체 content의 앞, 뒤로 있는 공백을 없앱니다.
*   처리가 잘 되었는지 확인을 위해 상위 5개의 결과값을 출력합니다.





In [11]:
total['content'] = total['content'].str.replace(r'[^a-zA-Z0-9가-힣\s]', ' ')
total['content'] = total['content'].str.replace(r'[ ]{2,}', ' ')
total['content'] = total['content'].str.strip()

total['content'][:5]

0    말벗 되고 동화 들려주고 서울 자치구 AI 보급 늘어치매 예방 정보 알림 등 효과에...
1    여당 4050 응집력 발휘 기대 야당은 2030에 공들여전문가 점점 분산 투표 성격...
2    현대차 프리미엄 브랜드 제네시스가 31일 전기차 기반의 GT 콘셉트카인 제네시스 엑...
3    개인통산 다섯번째 우승 대기록 위기 많았지만 똘똘 뭉쳐 극복이런 기회 다시 올까 싶...
4    한국마사회가 공공기관 고객만족도 PCSI 조사를 조작하는 등 불법행위를 저질렀다는 ...
Name: content, dtype: object

### **최소 5개 사례 print하여 제시**

*   head 함수를 통해 출력합니다. 저는 10개를 출력해 보았습니다.



In [12]:
total.head(10)

,title,date,url,content
0,‘대면 복지’ 빈자리 채우는 ‘돌봄 로봇’,2021. 04. 01,http://news.khan.co.kr/kh_news/khan_art_view.h...,말벗 되고 동화 들려주고 서울 자치구 AI 보급 늘어치매 예방 정보 알림 등 효과에...
1,보수 야당도 독려·투표율 의미 약화…달라진 ‘사전투표 공식’,2021. 04. 01,http://news.khan.co.kr/kh_news/khan_art_view.h...,여당 4050 응집력 발휘 기대 야당은 2030에 공들여전문가 점점 분산 투표 성격...
2,'영끌' 해서라도 사고 싶다... 제네시스 전기차 기반 GT 콘셉트카 '엑스',2021. 03. 31,http://news.khan.co.kr/kh_news/khan_art_view.h...,현대차 프리미엄 브랜드 제네시스가 31일 전기차 기반의 GT 콘셉트카인 제네시스 엑...
3,‘정규리그 우승’ 전창진 감독 “날 선택해준 KCC에 보답하고 싶었다”,2021. 03. 31,http://news.khan.co.kr/kh_news/khan_art_view.h...,개인통산 다섯번째 우승 대기록 위기 많았지만 똘똘 뭉쳐 극복이런 기회 다시 올까 싶...
4,"감사원, 공공기관 고객만족도 조사 조작 등 마사회 불법행위 확인",2021. 03. 30,http://news.khan.co.kr/kh_news/khan_art_view.h...,한국마사회가 공공기관 고객만족도 PCSI 조사를 조작하는 등 불법행위를 저질렀다는 ...
5,"[단독]감사원, 마사회 공익제보자 직위해제 부당 판단…제보자 “사측 압박 진행형”",2021. 03. 30,http://news.khan.co.kr/kh_news/khan_art_view.h...,감사원이 30일 발표한 한국마사회 감사 결과에는 마사회가 사측의 부정행위를 언론에 ...
6,"김준형 ""한미관계 가스라이팅 상태…동맹중독도""",2021. 03. 30,http://news.khan.co.kr/kh_news/khan_art_view.h...,김준형 국립외교원장이 30일 펴낸 저서에서 한 미동맹에 대한 과도한 의존을 가스라이...
7,"마사회, 가족·고객 동원해 만족도 조작했다",2021. 03. 30,http://news.khan.co.kr/kh_news/khan_art_view.h...,감사원 부정행위 확인 암행 감독관 사진 공유 적발 막기도 사측의 공익제보자 직위해제...
8,"KCC, 5년 만에 정규리그 1위…‘왕조 재건’ 도전",2021. 03. 30,http://news.khan.co.kr/kh_news/khan_art_view.h...,모비스가 DB에 지면서 확정전창진 감독 3개 팀서 1위 달성전주 KCC가 5년 만에...
9,"여의도 벚꽃길 내달 2일부터 사전신청…영등포구, ""절차 투명공개·양도 등 금지""",2021. 03. 29,http://news.khan.co.kr/kh_news/khan_art_view.h...,서울 영등포구는 벚꽃감상을 위해 윤중로 여의서로 를 찾으려는 시민들을 대상으로 오는...


### **CSV 파일로 저장**

*   google drive를 사용하여 제 드라이브의 폴더에 경향_crawling이라는 파일 이름으로 csv 파일을 저장하였습니다.



In [13]:
total.to_csv('/content/drive/MyDrive/Colab Notebooks/경향_crawling.csv', encoding="utf-8-sig")